<a href="https://colab.research.google.com/github/mbatalha-svc/CAPES_WebScraper/blob/main/DadosCAPES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo
Esse trabalho tem como objetivo a extração de títulos a partir de uma pesquisa por termos na plataforma do catálogo de teses da CAPES.

https://catalogodeteses.capes.gov.br/catalogo-teses/#!/

In [7]:
search_terms = "\"PRÁTICAS PEDAGÓGICAS\""
search_terms = input("Insira o texto da pesquisa: ")

#Código

In [1]:
# USAR SELENIUM PARA ACESSAR O LINKEDIN
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 5.2 MB/s 
     |████████████████████████████████| 358 kB 58.0 MB/s 
     |████████████████████████████████| 139 kB 27.0 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 4.1 MB 47.6 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.10 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Hit:1 https://

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.common.exceptions import WebDriverException
import time

In [3]:
def getChromeWebDriver():
  chrome_options = webdriver.ChromeOptions()

  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chrome_options.add_argument("--window-size=2560,1440")

  wd_chrome = webdriver.Chrome(options = chrome_options)

  return wd_chrome

In [4]:
wd = getChromeWebDriver()
wd.get("https://catalogodeteses.capes.gov.br/catalogo-teses/#!/")

In [5]:
wd.title

'Catálogo de Teses & Dissertações - CAPES'

In [6]:
wd.page_source

'<html><head><style type="text/css">[uib-typeahead-popup].dropdown-menu{display:block;}</style><style type="text/css">.uib-time input{width:50px;}</style><style type="text/css">[uib-tooltip-popup].tooltip.top-left > .tooltip-arrow,[uib-tooltip-popup].tooltip.top-right > .tooltip-arrow,[uib-tooltip-popup].tooltip.bottom-left > .tooltip-arrow,[uib-tooltip-popup].tooltip.bottom-right > .tooltip-arrow,[uib-tooltip-popup].tooltip.left-top > .tooltip-arrow,[uib-tooltip-popup].tooltip.left-bottom > .tooltip-arrow,[uib-tooltip-popup].tooltip.right-top > .tooltip-arrow,[uib-tooltip-popup].tooltip.right-bottom > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.top-left > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.top-right > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.bottom-left > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.bottom-right > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.left-top > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.left-bottom > .tooltip-arrow,[uib-toolt

In [8]:
search_box = wd.find_element(By.CSS_SELECTOR, "[ng-model=\"termo\"]")
search_box.clear()
search_box.send_keys(search_terms)

submit_button = wd.find_element(By.CLASS_NAME, "btn-primary")
submit_button.click()
time.sleep(3)

In [9]:
def getNextButton(wd):
  next_page = wd.find_element(By.CLASS_NAME, "pagination-next")

  if "disabled" not in next_page.get_attribute("class"):
    return next_page.find_element(By.TAG_NAME, "a")
  else:
    return None

In [10]:
def isActuallyClickable (wd, element):
  def __call__(wd):
    try:
      #wd.execute_script('console.log("clicking");')
      print("CLIQUEI")
      element.click()
    except WebDriverException as e:
      print("Tentei clicar e não deu")
      return False
    except UnexpectedAlertPresentException:
      wd.switch_to.alert().accept()
      return False
    #wd.execute_script('console.log("clickable");')
    return True
  return __call__

In [ ]:
source_log = []

res_list = []

count_pages = 0
cur_value = 0
while True:
  while True:
    #source_log.append(wd.page_source)
    #count_pages = count_pages + 1
    #print(count_pages)
    # do something with the data on the page
    try:
      info = wd.find_element(By.CLASS_NAME, "text-info")
      page_value = int(info.text.replace(' ', '-').split('-')[1])
      max_value = int(info.text.replace(' ', '-').split('-')[4])
      print(f"START {page_value} FROM {cur_value} OF {max_value}")
      WebDriverWait(wd, 15).until(EC.text_to_be_present_in_element((By.CLASS_NAME, "resultados"), f"{str(page_value)}.\n"))
      if page_value > cur_value:
        results = wd.find_element(By.CLASS_NAME, "resultados")
        parsed = results.text.split("\n")
        print(f"STARTED {parsed[0]}")
        res_list.extend(parsed)
        cur_value = page_value
      if cur_value > (max_value - 20):
        break
      # call next page
      next = getNextButton(wd)
    
      if next != None:
        WebDriverWait(wd, 180).until(EC.none_of(EC.visibility_of(wd.find_element(By.CLASS_NAME, "block-ui-overlay")), 
                                               EC.element_to_be_clickable((By.CLASS_NAME, "block-ui-overlay"))))
        WebDriverWait(wd, 180).until(isActuallyClickable(wd, next))
    except UnexpectedAlertPresentException:
      print("erro foi acionado")
      wd.switch_to.alert().accept()
  if cur_value > (max_value - 20):
    break

START 1 FROM 0 OF 8685
STARTED 1.
CLIQUEI
START 21 FROM 1 OF 8685
STARTED 21.
CLIQUEI
START 41 FROM 21 OF 8685
STARTED 41.
CLIQUEI
START 61 FROM 41 OF 8685
STARTED 61.
CLIQUEI
START 81 FROM 61 OF 8685
STARTED 81.
CLIQUEI
START 101 FROM 81 OF 8685
STARTED 101.
CLIQUEI
START 121 FROM 101 OF 8685
STARTED 121.
CLIQUEI
START 141 FROM 121 OF 8685
STARTED 141.
CLIQUEI
START 161 FROM 141 OF 8685
STARTED 161.
CLIQUEI
START 181 FROM 161 OF 8685
STARTED 181.
CLIQUEI
START 201 FROM 181 OF 8685
STARTED 201.
CLIQUEI
START 221 FROM 201 OF 8685
STARTED 221.
CLIQUEI
START 241 FROM 221 OF 8685
STARTED 241.
CLIQUEI
START 261 FROM 241 OF 8685
STARTED 261.
CLIQUEI
START 281 FROM 261 OF 8685
STARTED 281.
CLIQUEI
START 301 FROM 281 OF 8685
STARTED 301.
CLIQUEI
START 321 FROM 301 OF 8685
STARTED 321.
CLIQUEI
START 341 FROM 321 OF 8685
STARTED 341.
CLIQUEI
START 361 FROM 341 OF 8685
STARTED 361.
CLIQUEI
START 381 FROM 361 OF 8685
STARTED 381.
CLIQUEI
START 401 FROM 381 OF 8685
STARTED 401.
CLIQUEI
START 421 FR

In [ ]:
res_list

In [ ]:
import pandas as pd

In [ ]:
search_data = pd.DataFrame(columns = ['search', 'index', 'description'])

In [ ]:
clean_results = list(filter(lambda a:a!="Detalhes" and a!="Trabalho anterior à Plataforma Sucupira", res_list))

In [ ]:
clean_results[len(clean_results) - 2]

In [ ]:
index = []
description = []
for i in range(0, len(clean_results)):
  if i % 2 == 0:
    index.append(clean_results[i])
  else:
    description.append(clean_results[i])
  
  i = i + 1

In [ ]:
for i in range(0, len(index)):
  search_data = search_data.append({'search': search_terms, 'index': index[i], 'description': description[i]}, 
                                   ignore_index=True)

In [ ]:
pd.options.display.max_colwidth = 1000

In [ ]:
desc_len = search_data.description.apply(lambda desc: len(desc))
desc_len.max()

In [ ]:
search_data

In [ ]:
search_data.to_csv(f"/content/{search_terms}.csv", index=False)